In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [4]:
filename = "all_arxiv_titles.csv"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

#simple embedding scheme
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i,c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)

seq_length = 25
dataX = []
dataY = []

for line in raw_text.splitlines()[:]:
    for i in range(0, len(line)- seq_length, 1):
        prefix = line[i: i + seq_length]
        suffix = line[i+seq_length]
        dataX.append([char_to_int[char] for char in prefix])
        dataY.append([char_to_int[suffix]])
        
n_patterns = len(dataX)

X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X /float(n_vocab)
y = torch.tensor(dataY)

y = y.reshape(-1)

In [6]:
print(y.shape)

torch.Size([565976])


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from sklearn.model_selection import train_test_split

class CharModel(nn.Module):
    def __init__(self, hidden_size=128):
        super(CharModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(hidden_size, n_vocab)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # take only the last output
        x = self.linear(self.dropout(x))  # produce output
        return x

# Parameters
n_epochs = 40
batch_size = 128
hidden_size = 256

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoaders
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
test_loader = data.DataLoader(data.TensorDataset(X_test, y_test), shuffle=False, batch_size=batch_size)

# Initialize model, optimizer, and loss function
model = CharModel(hidden_size=hidden_size)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")

best_model = None
best_loss = np.inf

# Training loop
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader.dataset)
    
    # Validation
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            test_loss += loss.item()
            
    test_loss /= len(test_loader.dataset)
    
    if test_loss < best_loss:
        best_loss = test_loss
        best_model = model.state_dict()
        
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

# Save the best model and char_to_int dictionary
torch.save([best_model, char_to_int], "single-char.pth")


Epoch 1: Train Loss: 3.0071, Test Loss: 2.9146
Epoch 2: Train Loss: 2.9033, Test Loss: 2.8718
Epoch 3: Train Loss: 2.8716, Test Loss: 2.8379
Epoch 4: Train Loss: 2.8389, Test Loss: 2.8008
Epoch 5: Train Loss: 2.8092, Test Loss: 2.7720
Epoch 6: Train Loss: 2.7868, Test Loss: 2.7472
Epoch 7: Train Loss: 2.7703, Test Loss: 2.7310
Epoch 8: Train Loss: 2.7554, Test Loss: 2.7156
Epoch 9: Train Loss: 2.7424, Test Loss: 2.7017
Epoch 10: Train Loss: 2.7311, Test Loss: 2.6881
Epoch 11: Train Loss: 2.7221, Test Loss: 2.6782
Epoch 12: Train Loss: 2.7119, Test Loss: 2.6671
Epoch 13: Train Loss: 2.7023, Test Loss: 2.6557
Epoch 14: Train Loss: 2.6950, Test Loss: 2.6488
Epoch 15: Train Loss: 2.6864, Test Loss: 2.6404
Epoch 16: Train Loss: 2.6806, Test Loss: 2.6319
Epoch 17: Train Loss: 2.6729, Test Loss: 2.6248
Epoch 18: Train Loss: 2.6681, Test Loss: 2.6191
Epoch 19: Train Loss: 2.6630, Test Loss: 2.6136
Epoch 20: Train Loss: 2.6570, Test Loss: 2.6232
Epoch 21: Train Loss: 2.6517, Test Loss: 2.6050
E

In [12]:
def generate_sequence(model, seed, n_chars, char_to_int, int_to_char, device='cpu'):
    model.eval()
    seed = torch.tensor([char_to_int[c] for c in seed], dtype=torch.long).unsqueeze(0).to(device)
    hidden = None
    generated = seed.tolist()[0]  # Convert tensor to list

    with torch.no_grad():
        for _ in range(n_chars):
            # Prepare input for the model
            input_seq = seed.float().unsqueeze(-1)  # Ensure the input has shape (batch_size, seq_len, input_size)
            output, hidden = model.lstm(input_seq, hidden)
            output = model.dropout(output)
            output = model.linear(output[:, -1, :])  # Take the output of the last time step

            # Sample from the output distribution
            probs = torch.nn.functional.softmax(output, dim=1)
            next_char = torch.multinomial(probs, 1).item()
            generated.append(next_char)

            # Update seed for the next iteration
            seed = torch.tensor([[next_char]], dtype=torch.long).to(device)
            
    # Convert the generated sequence back to characters
    generated_chars = ''.join([int_to_char[i] for i in generated])
    return generated_chars

int_to_char = dict((i,c) for i,c in enumerate(chars))
# Load the best model
model.load_state_dict(best_model)

# Generate a sequence
seed = 'the advantages of'
n_chars = 100  # Number of characters to generate
generated_text = generate_sequence(model, seed, n_chars, char_to_int, int_to_char)
print(generated_text)


the advantages of@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
